# Partitioning the input files

ROMS requires partitioned (or tiled) input files so that the simulation can be parallelized over multiple nodes. `ROMS-Tools` can create these partitioned files for you.

## Writing some example files
First we make some (non-partitioned) example files, which we will then partition in the next section. 

### Example grid file

In [1]:
from roms_tools import Grid

In [2]:
grid = Grid(
    nx=300,
    ny=150,
    size_x=23000,
    size_y=12000,
    center_lon=-161.0,
    center_lat=14.4,
    rot=-3.0,
)

In [3]:
grid.ds

<xarray.Dataset> Size: 5MB
Dimensions:       (eta_rho: 152, xi_rho: 302, xi_u: 301, eta_v: 151,
                   eta_coarse: 77, xi_coarse: 152, s_rho: 100, s_w: 101)
Coordinates:
    lat_rho       (eta_rho, xi_rho) float64 367kB -46.17 -46.0 ... 40.06 39.92
    lon_rho       (eta_rho, xi_rho) float64 367kB 110.6 111.3 ... 313.9 314.5
    lat_u         (eta_rho, xi_u) float64 366kB -46.08 -45.91 ... 40.13 39.99
    lon_u         (eta_rho, xi_u) float64 366kB 110.9 111.6 ... 313.6 314.2
    lat_v         (eta_v, xi_rho) float64 365kB -45.94 -45.77 ... 39.82 39.68
    lon_v         (eta_v, xi_rho) float64 365kB 110.5 111.1 ... 313.8 314.4
    lat_coarse    (eta_coarse, xi_coarse) float64 94kB -46.48 -46.14 ... 40.08
    lon_coarse    (eta_coarse, xi_coarse) float64 94kB 110.4 111.7 ... 314.9
Dimensions without coordinates: eta_rho, xi_rho, xi_u, eta_v, eta_coarse,
                                xi_coarse, s_rho, s_w
Data variables: (12/15)
    angle         (eta_rho, xi_rho) float64 367kB 0.3739 0.3739 ... -0.3063
    f             (eta_rho, xi_rho) float64 367kB -0.0001049 ... 9.333e-05
    pm            (eta_rho, xi_rho) float64 367kB 1.838e-05 ... 1.838e-05
    pn            (eta_rho, xi_rho) float64 367kB 1.936e-05 ... 1.936e-05
    spherical     |S1 1B b'T'
    mask_rho      (eta_rho, xi_rho) int32 184kB 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0
    ...            ...
    mask_coarse   (eta_coarse, xi_coarse) int32 47kB 1 1 1 1 1 1 ... 0 0 0 0 0 0
    h             (eta_rho, xi_rho) float64 367kB 4.082e+03 4.082e+03 ... 5.0
    sigma_r       (s_rho) float32 400B -0.995 -0.985 -0.975 ... -0.015 -0.005
    Cs_r          (s_rho) float32 400B -0.992 -0.9753 ... -8.89e-05 -9.874e-06
    sigma_w       (s_w) float32 404B -1.0 -0.99 -0.98 -0.97 ... -0.02 -0.01 0.0
    Cs_w          (s_w) float32 404B -1.0 -0.9837 -0.9667 ... -3.95e-05 0.0
Attributes: (12/13)
    title:                   ROMS grid created by ROMS-Tools
    roms_tools_version:      2.6.3.dev44+g780e06f.d20250623
    size_x:                  23000
    size_y:                  12000
    center_lon:              -161.0
    center_lat:              14.4
    ...                      ...
    straddle:                False
    topography_source_name:  ETOPO5
    hmin:                    5.0
    theta_s:                 5.0
    theta_b:                 2.0
    hc:                      300.0

In [4]:
filepath_grid = "/pscratch/sd/n/nloose/grids/my_roms_grid.nc"

In [5]:
grid.save(filepath_grid)

INFO - Writing the following NetCDF files:
/pscratch/sd/n/nloose/grids/my_roms_grid.nc


[PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.nc')]

### Example surface forcing file

In [6]:
from roms_tools import SurfaceForcing
from datetime import datetime

In [7]:
start_time = datetime(2011, 1, 1)
end_time = datetime(2011, 1, 2)

In [8]:
era5_path = "/global/cfs/projectdirs/m4746/Datasets/ERA5/GLOBAL/reanalysis-era5-single-levels_2011-01.nc"

In [9]:
surface_forcing = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": era5_path},
    use_dask=True,
)

INFO - Data will be interpolated onto fine grid.
INFO - Applying 2D horizontal fill to the source data before regridding.
INFO - Applying 2D horizontal fill to the source data before regridding.


In [10]:
surface_forcing.ds

<xarray.Dataset> Size: 32MB
Dimensions:   (time: 25, eta_rho: 152, xi_rho: 302)
Coordinates:
    number    int64 8B 0
    expver    (time) <U4 400B dask.array<chunksize=(1,), meta=np.ndarray>
    abs_time  (time) datetime64[ns] 200B 2011-01-01 ... 2011-01-02
  * time      (time) float64 200B 4.018e+03 4.018e+03 ... 4.019e+03 4.019e+03
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    uwnd      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    vwnd      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    swrad     (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    lwrad     (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    Tair      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    rain      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
    qair      (time, eta_rho, xi_rho) float32 5MB dask.array<chunksize=(1, 152, 302), meta=np.ndarray>
Attributes:
    title:                 ROMS surface forcing file created by ROMS-Tools
    roms_tools_version:    2.6.3.dev44+g780e06f.d20250623
    start_time:            2011-01-01 00:00:00
    end_time:              2011-01-02 00:00:00
    source:                ERA5
    correct_radiation:     True
    wind_dropoff:          False
    use_coarse_grid:       False
    model_reference_date:  2000-01-01 00:00:00
    type:                  physics

In [11]:
filepath_forcing = "/pscratch/sd/n/nloose/forcing/my_surface_forcing.nc"

In [12]:
surface_forcing.save(filepath_forcing)

INFO - Writing the following NetCDF files:
/pscratch/sd/n/nloose/forcing/my_surface_forcing_201101.nc


[########################################] | 100% Completed | 304.59 s


[PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing_201101.nc')]

## Partitioning existing files
We will now partition the two files written in the previous section. However, note that the `partition_netcdf` tool in `ROMS-Tools` operates independently of whether the files were originally created using `ROMS-Tools`.

In [13]:
from roms_tools import partition_netcdf

We need to tell the `partition_netcdf` function what domain decomposition to use via the following two parameters:

* `np_eta` : The number of partitions along the `eta` direction (corresponding to `ny` in the grid).
* `np_xi` : The number of partitions along the `xi` direction (corresponding to `nx` in the grid).

In [14]:
%time partition_netcdf(filepath_grid, np_eta=3, np_xi=6)

CPU times: user 310 ms, sys: 60 ms, total: 370 ms
Wall time: 420 ms


[PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.00.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.01.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.02.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.03.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.04.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.05.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.06.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.07.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.08.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.09.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.13.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.14.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.15.nc'),
 PosixPa

In [15]:
%time partition_netcdf(filepath_forcing, np_eta=3, np_xi=6)

CPU times: user 53.1 s, sys: 27.4 s, total: 1min 20s
Wall time: 1min 20s


[PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.00.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.01.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.02.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.03.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.04.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.05.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.06.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.07.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.08.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.09.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/forcing/my_surface_forcing.13.nc'),
 PosixPath('/pscratc

## Providing a list of files

The `partition_netcdf` function also accepts a list of input files, allowing you to partition multiple NetCDF files in a single call. 

In [16]:
%time partition_netcdf([filepath_grid, filepath_forcing], np_eta=3, np_xi=6)

CPU times: user 54.1 s, sys: 32.5 s, total: 1min 26s
Wall time: 1min 26s


[PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.00.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.01.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.02.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.03.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.04.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.05.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.06.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.07.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.08.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.09.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.13.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.14.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.15.nc'),
 PosixPa

## Specifying an output directory for partitioned files

The `partition_netcdf` function accepts an optional `output_dir` parameter, which allows you to control where the partitioned files are saved.

* If `output_dir` is set to `None` (the default), partitioned files are saved in the same directory as the input file.
* If an explicit `output_dir` is provided, all partitioned files will be written to that location.

In the examples above, we used the default behavior. Now let's specify a custom `output_dir` to save the output files elsewhere.

In [17]:
%time partition_netcdf(filepath_grid, np_eta=3, np_xi=6, output_dir='/pscratch/sd/n/nloose/grids/partitioned')

CPU times: user 315 ms, sys: 56.1 ms, total: 371 ms
Wall time: 423 ms


[PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.00.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.01.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.02.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.03.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.04.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.05.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.06.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.07.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.08.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.09.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/partitioned/my_roms_grid.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/grid

## Ignoring Coarse Dimensions During Partitioning

The grid file generated by `ROMS-Tools` always includes the dimensions `eta_coarse` and `xi_coarse`. These dimensions, along with their associated variables, are only relevant when **coarse-resolution forcing** is used. This typically applies when surface forcing fields are generated on a grid coarser than the ROMS model grid, see the [ROMS-Tools documentation on surface forcing](https://roms-tools.readthedocs.io/en/latest/surface_forcing.html#Fine-vs.-coarse-grid). The coarse forcing configuration is activated when the ROMS `bulk_frc.opt` file includes the setting:
```bash
interp_frc = 1
```
With this option enabled, ROMS interpolates surface forcing data from the coarse grid onto the fine model grid at runtime.

If your simulation does **not** use coarse forcing, then the `eta_coarse` and `xi_coarse` dimensions in the grid file can be safely ignored, including during partitioning. This allows for a broader range of valid values for `np_eta` and `np_xi`, since only the fine grid dimensions must be divisible by these parameters. In contrast, if the coarse dimensions `eta_coarse`, `xi_coarse` are included, **both** fine and coarse grid sizes must be divisible by `np_eta` and `np_xi`.

The `partition_netcdf()` function provides a parameter `include_coarse_dimensions` to control this behavior.

### Example
Our grid above was created with:

* `nx = 300` (corresponding to `xi_rho`)
* `ny = 150` (corresponding to `eta_rho`)

By default, ROMS-Tools also includes coarse grid dimensions:

* `nx / 2 = 150` (corresponding to `xi_coarse`)
* `ny / 2 = 75` (corresponding to `eta_coarse`)

Now let’s try partitioning with:

* `np_xi = 12`
* `np_eta = 6`

These values divide the fine dimensions:

* `np_xi = 12` divides `nx = 300`
* `np_eta = 6` divides `ny = 150`

However, they do not divide the coarse dimensions:

* 12 does not divide 150

* 6 does not divide 75

As a result, if `include_coarse_dims = True` (the default), partitioning will fail with an error like:

In [18]:
%time partition_netcdf(filepath_grid, np_eta=6, np_xi=12, include_coarse_dims=True)

ValueError: Dimension 'eta_coarse' of size 75 cannot be evenly divided into 6 partitions.

To avoid this, you can set `include_coarse_dims = False` to skip partitioning along the coarse dimensions:

In [19]:
%time partition_netcdf(filepath_grid, np_eta=6, np_xi=12, include_coarse_dims=False)

CPU times: user 1.25 s, sys: 240 ms, total: 1.49 s
Wall time: 1.71 s


[PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.00.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.01.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.02.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.03.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.04.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.05.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.06.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.07.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.08.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.09.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.13.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.14.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.15.nc'),
 PosixPa